This is the code for the data extractor.

First, some maintenance work.

In [44]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

#let python display all columns on output.
pd.set_option('display.max_columns', None)

File_Assember( ):
Read one or multiple concatenated files into a dataframe.

Input: file path.

Output: A pandas dataframe.

In [45]:
#set the file path, users can change to their own paths
path = r'C:\Users\Eric Zhong\Documents\2.INTERNSHIP\RESEARCH\iFiles_for_Leo'

#The function passes in the file path and file_no_, which means the number of file in order. 
#For example, we want to read the 5th txt file only: File_Assembler(path_, 5, 5)
#If we want to read files 1 - 7: File_Assembler(path_, 1, 7)
#in total 58 txt files for now

def File_Assembler(path_, file_lwr_, file_upper_):
    all_files = glob.glob(path_ + "/*.txt")
    count = 0
    #create an empty array, and use a for loop to append files to the array.
    li = []
    for filename in all_files:
        count = count + 1
        if count >= file_lwr_ and count <= file_upper_:
            x = pd.read_csv(filename, index_col=None, header=0, sep = '\t')
            li.append(x)
    Assembled_File = pd.concat(li, axis=0, ignore_index=True)
    return Assembled_File

File_Loader( )
Used when the user only wants to load one file as an alternative

INPUT:
    File path
    The number of file

OUTPUT:
    Dataframe containing the data


In [124]:
#Since now the scenario has changed, File_assembler is no longer needed at most times. A File_loader is written below
#in order to load single files more efficiently. User passes in the path and also the order number of file.
def File_Loader(path_, file_no_):
    all_files = glob.glob(path_ + "/*.txt")
    count = 0
    for filename in all_files:
        count = count + 1
        if count == file_no_:
            file_ = pd.read_csv(filename, index_col=None, header=0, sep = '\t')
    return file_

Engine Parameters:
    The function that helps calculate some engine parameters

INPUT:
    EVO
    EVC
    Number of strokes
OUTPUT:
    Engine cycle period
    Sampling window width
    Sampling window offset

In [ ]:
#according to the papers:
            #Window_Width = EVC - EVO
    
#There is an "offset", which is the CAD before sampling window takes place in the first cycle: 
            #Offset = Total_Crank_Rotation - Window_Width

#Period of the cyclic sampling process, which is simply the total crank rotation in a full engine cycle:
            #Period = Total_Crank_Rotation = (Stroke_Number / 2) * 360
def Engine_Parameters(EVO_Degree_, EVC_Degree_, Stroke_Number_):
    Period_ = (Stroke_Number_ / 2) * 360
    Window_Width_ = EVC_Degree_ - EVO_Degree_
    Window_Offset_ = Period_ - Window_Width_
    return Period_, Window_Width_, Window_Offset_
#With our engine as example: 
#EVO = 128
#EVC = 382
#Stroke_Number = 4
#User can use the function like this:
(Period, Window_Width, Offset) = Engine_Parameters(128, 382, 4)

Filter_NONOX( ):
Input
    Data from File_Assembler or File_Loader.

Output
    Dataframe that contains only Crank Angle, NO, NOx.

In [5]:
def Filter_NONOX(Assembled_Data):
    # drop all unneccessary columns, and the units row.
    Assembled_Data_filtered = Assembled_Data[['Crank Angle', 'NO', 'NOx']].drop([0])
    # change data types from strings to floats
    Assembled_Data_filtered = Assembled_Data_filtered.astype('float64')
    # drop all rows with 0 values
    Assembled_Data_filtered = Assembled_Data_filtered[Assembled_Data_filtered.NO != 0.0000]
    #reset the index and drop original index column
    Assembled_Data_filtered = Assembled_Data_filtered.reset_index().drop(['index'], axis=1)

    #Change Crank Angle to a continuous series, rather than -360 to 3600 for each file.
    Assembled_Data_filtered['Crank Angle'] = Assembled_Data_filtered.index
    return Assembled_Data_filtered

NONOX_Cyclic_Avg( ):
    A funtion that calculates the average of NO NOx within sampling window.

INPUT:
    NONOX filtered dataframe
    Sampling window width
    Sampling window offset
    Engine cycle period
OUTPUT: 
    A dataframe containing average value of NO, NOx for each cycle.

In [6]:
def NONOX_Cyclic_Avg(NONOXDataframe_, Window_Width_, Offset_, Period_):
    Cyclic_Avg_ = pd.DataFrame([], columns = ['NO_Avg', 'NOx_Avg'])
    x = 0
    while (Offset_ + x * Period_ < NONOXDataframe_.shape[0]):
        Cyclic_Avg_.loc[x+1, 'NO_Avg'] = (NONOXDataframe_['NO'].iloc[Offset_ + Period_ * x : Offset_ + Period_ * x + Window_Width_].mean())
        Cyclic_Avg_.loc[x+1, 'NOx_Avg'] = (NONOXDataframe_['NOx'].iloc[Offset_ + Period_ * x : Offset_ + Period_ * x + Window_Width_].mean())
        x = x + 1
    return Cyclic_Avg_

NO_NOx_plotter( )
Plots NO NOx data against crank angle.

Input:
    The NONOx filtered dataframe.
    Start and end CAD he wishes to plot, by default 0 - 4319999.
    Whether to plot NO, NOx or both
    y lower and upper limit, 0 and 9 by default, which is the range of the entire dataframe.

Output: NO NOX plot.

In [ ]:
def NONOX_plotter(Filtered_data_, Start_Crank_Angle_ = 0, End_Crank_Angle_ = 4319999, plot_NO = True, plot_NOx = True, y_lower_limit_ = 0, y_upper_limit_ = 9):
    axes = plt.gca()
    if plot_NO == True and plot_NOx == True:
        Filtered_data_.iloc[Start_Crank_Angle_ : End_Crank_Angle_].plot(kind='line', x='Crank Angle', y='NO', ylim = (y_lower_limit_ , y_upper_limit_), ax=axes)
        Filtered_data_.iloc[Start_Crank_Angle_ : End_Crank_Angle_].plot(kind='line', x='Crank Angle', y='NOx', ylim = (y_lower_limit_ , y_upper_limit_), ax=axes)
    elif plot_NO == True and plot_NOx == False:
        Filtered_data_.iloc[Start_Crank_Angle_ : End_Crank_Angle_].plot(kind='line', x='Crank Angle', y='NO', ylim = (y_lower_limit_ , y_upper_limit_), ax=axes)
    else:
        Filtered_data_.iloc[Start_Crank_Angle_ : End_Crank_Angle_].plot(kind='line', x='Crank Angle', y='NOx', ylim = (y_lower_limit_ , y_upper_limit_), ax=axes)
    plt.show()
#as an example, we want to plot from "engine", from 205000 CAD to 215000 degree, the variation of NOx only:
#NO_NOx_plotter(engine, 205000, 215000, False, True)
#or, as False = 0, and True = 1:
#NO_NOx_plotter(engine, 205000, 215000, 0, 1)

NONOX_separator( ):
A function that is able to separate data within sampling window and data out of sampling window.

Input:
    NONOX filtered dataframe
    Window width
    Window offset
    Engine cycle period
Output:
    Dataframe containing data within sampling window.
    Dataframe containing the other data.

In [ ]:
def NONOX_separator(NONOXDataframe_, Window_Width_, Offset_, Period_):
    Data_Within_Window_ = pd.DataFrame([], columns = ['Crank Angle', 'NO', 'NOx'])
    x = 0
    while (Offset_ + x * Period_ < NONOXDataframe_.shape[0]):
            samples = NONOXDataframe_.iloc[(Offset_ + x * Period_) : (Offset_ + x * Period_ + Window_Width_)]
            Data_Within_Window_ = Data_Within_Window_.append(samples)
            x = x + 1
    Data_Outside_Window_ = pd.DataFrame([], columns = ['Crank Angle', 'NO', 'NOx'])
    Data_Outside_Window_ = NONOXDataframe_[~NONOXDataframe_.index.isin(Data_Within_Window_.index)]
    
    return Data_Within_Window_, Data_Outside_Window_

In [ ]:
############################################## PRESSURE FUNCTIONS FROM NOW ON##############################################

Filter_PCYL1( ):

Input:
    Data from File_Assembler or File_Loader.

Output: 
    Dataframe that contains only Crank Angle, PCYL1.

In [122]:
def Filter_PCYL1(Assembled_Data):
    # drop all unneccessary columns, and the units row.
    Assembled_Data_filtered = Assembled_Data[['Crank Angle', 'PCYL1']].drop([0])
    # change data types from strings to floats
    Assembled_Data_filtered = Assembled_Data_filtered.astype('float64')
    #reset the index and drop original index column
    Assembled_Data_filtered = Assembled_Data_filtered.reset_index().drop(['index'], axis=1)
    #Make Crank Angle start at 0 instead of -360
    Assembled_Data_filtered['Crank Angle'] = Assembled_Data_filtered['Crank Angle'] + 360

    return Assembled_Data_filtered

PMAX( ):
Extracts cyclic maximum PCYL1.

INPUT:
    PCYL1 filtered dataframe
    Offset of engine cycle (how much CAD does the engine go through before the first recorded cycle)
    Engine cycle period
OUTPUT:
    A dataframe that contains maximum PCYL1 for every engine cycle

In [8]:
def PMAX(P_Dataframe_, EngineCycle_Offset_, Period_):
    PMAX_ = pd.DataFrame([], columns = ['PMAX'])
    x = 0
    
    #Find the PMAX per degree, since there are multiple Ps per degree.
    PMax_Per_Degree_ = P_Dataframe_.groupby(['Crank Angle']).max()
    while ((EngineCycle_Offset_ + x * Period_) < PMax_Per_Degree_.shape[0]):
        #Find the max pressure value for every 720 degree cycle, append to the PMAX_ dataframe.
        PMAX_.loc[x+1] = list(PMax_Per_Degree_.loc[x * Period_: (x + 1) * Period_ - 1].max())
        #increment x
        x = x + 1
    #Drop the last value of PMAX_, as there were originally 432001 records instead of 432000, need further clarification
    #PMAX_ = PMAX_.drop(PMAX_.shape[0])
    return PMAX_
#The output dataframe should have its index as the number of cycles.